## 1 - Import

In [6]:
import os
import tiktoken
import difflib
import pandas as pd
import io
import sys
from dotenv import load_dotenv
from firecrawl import FirecrawlApp
from langchain_openai import AzureChatOpenAI
from langchain.embeddings import OpenAIEmbeddings, AzureOpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
import requests
import urllib.parse
import fitz


In [7]:
load_dotenv()

True

## 2 - Models

In [8]:
llm_4omini = AzureChatOpenAI(
    azure_endpoint=os.getenv('Azure_OpenAI_OB_Endpoint_4mini'), 
    openai_api_version="2024-10-21",   
    model_name="gpt-4o-mini",
    openai_api_key=os.getenv('Azure_OpenAI_OB_Key_4mini'), 
    openai_api_type="azure",
    temperature=0,
    deployment_name="gpt4o-mini")
    #streaming=True

In [9]:
embeddings = AzureOpenAIEmbeddings(
    model="text-embedding-3-large",
    azure_endpoint=os.getenv('Azure_OpenAI_emb_3_large_azure_endpoint'),
    openai_api_key=os.getenv('Azure_OpenAI_emb_3_large_api_key'),
    openai_api_version="2023-05-15",
    chunk_size=1000
)


/var/folders/ty/2g_hgfks4nl_84mz496fmn600000gn/T/ipykernel_55936/426928708.py:1: LangChainDeprecationWarning: The class `AzureOpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import AzureOpenAIEmbeddings``.
  embeddings = AzureOpenAIEmbeddings(


## 3 - Chunking

In [10]:
def count_tokens(text: str, model: str = "gpt-4o") -> int:
    encoding = tiktoken.encoding_for_model(model)
    tokens = encoding.encode(text)
    return len(tokens)

### 3.1 - Input url

In [11]:
url_aml_5="http://publications.europa.eu/resource/celex/32015L0849"
url_crr="http://publications.europa.eu/resource/celex/32013R0575"

In [12]:
url_aml_5_pdf="https://eur-lex.europa.eu/legal-content/FR/TXT/PDF/?uri=CELEX:32015L0849"
url_crr_pdf="https://eur-lex.europa.eu/legal-content/FR/TXT/PDF/?uri=CELEX:32013R0575"

### 3.2 - Url to local pdf

In [ ]:
def pipe_1_url_to_pdf(url, save_path):
    response = requests.get(url, 
                            headers={"Accept": "application/pdf"})# tres important pour ne pas renvoyer rdf ou autre format...
    with open(save_path, "wb") as f:
        f.write(response.content)

In [ ]:
pipe_1_url_to_pdf(url_aml_5_pdf,"/Users/oussa/Desktop/Github_perso/Advanced_RAG/data_notebooks/aml5.pdf")

In [ ]:
pipe_1_url_to_pdf(url_crr_pdf,
                     "/Users/oussa/Desktop/Github_perso/Advanced_RAG/data_notebooks/crr.pdf")

### 3.3 - PyMuPDF - Fitz

PyMuPDF is a high-performance Python library for data extraction, analysis, conversion & manipulation of PDF (and other) documents

In [21]:
def pipe_2_extract_text_with_pymupdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text.strip() 

In [22]:
url_aml_pdf_txt_fitz = pipe_2_extract_text_with_pymupdf("/Users/oussa/Desktop/Github_perso/Advanced_RAG/data_notebooks/aml5.pdf")

In [51]:
url_crr_pdf_txt_fitz = pipe_2_extract_text_with_pymupdf("/Users/oussa/Desktop/Github_perso/Advanced_RAG/data_notebooks/crr.pdf")

In [ ]:
count_tokens(url_aml_pdf_txt_fitz)

45232

In [52]:
count_tokens(url_crr_pdf_txt_fitz)

407103

## 3 - Chunking

### 3.1  - By Article

In [54]:
article_splitter = RecursiveCharacterTextSplitter(
    separators=   ["Article"],
    chunk_size= 100, 
# attention spliter	Si le « morceau » résultant dépasse chunk_size, 
# il redescend au séparateur suivant (plus petit).
    chunk_overlap=0)


In [62]:
chunks_aml5 = article_splitter.split_text(url_aml_pdf_txt_fitz)
chunks_aml5[:70]

["DIRECTIVES \nDIRECTIVE (UE) 2015/849 DU PARLEMENT EUROPÉEN ET DU CONSEIL \ndu 20 mai 2015 \nrelative à la prévention de l'utilisation du système financier aux fins du blanchiment de capitaux \nou du financement du terrorisme, modifiant le règlement (UE) no 648/2012 du Parlement européen \net du Conseil et abrogeant la directive 2005/60/CE du Parlement européen et du Conseil et la \ndirective 2006/70/CE de la Commission \n(Texte présentant de l'intérêt pour l'EEE) \nLE PARLEMENT EUROPÉEN ET LE CONSEIL DE L'UNION EUROPÉENNE, \nvu le traité sur le fonctionnement de l'Union européenne, et notamment son article 114, \nvu la proposition de la Commission européenne, \naprès transmission du projet d'acte législatif aux parlements nationaux, \nvu l'avis de la Banque centrale européenne (1), \nvu l'avis du Comité économique et social européen (2), \nstatuant conformément à la procédure législative ordinaire (3), \nconsidérant ce qui suit: \n(1)  \nLes flux d'argent illicite peuvent nuire à l'i

In [ ]:
chunks_crr = article_splitter.split_text(url_crr_pdf_txt_fitz)


In [64]:
chunks_crr[0:522]

['I \n(Actes législatifs) \nRÈGLEMENTS \nRÈGLEMENT (UE) N o 575/2013 DU PARLEMENT EUROPÉEN ET DU CONSEIL \ndu 26 juin 2013 \nconcernant les exigences prudentielles applicables aux établissements de crédit et aux entreprises \nd\'investissement et modifiant le règlement (UE) n o 648/2012 \n(Texte présentant de l\'intérêt pour l\'EEE) \nLE PARLEMENT EUROPÉEN ET LE CONSEIL DE L\'UNION EURO\xad\nPÉENNE, \nvu le traité sur le fonctionnement de l\'Union européenne, et \nnotamment son article 114, \nvu la proposition de la Commission européenne, \naprès transmission du projet d\'acte législatif aux parlements \nnationaux, \nvu l\'avis de la Banque centrale européenne ( 1 ), \nvu l\'avis du Comité économique et social européen ( 2 ), \nstatuant conformément à la procédure législative ordinaire, \nconsidérant ce qui suit: \n(1) \nDans sa déclaration du 2 avril 2009 sur le renforcement \ndu système financier, le G20 a appelé à agir de manière \ncohérente au niveau international pour renforcer la